In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split

2024-01-31 10:45:51.853530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Import data
df1 = pd.read_csv('cloudrm_clear.dat',skiprows=53, header=None, delimiter = ' ', skipinitialspace = True ,index_col = 0)
df2 = pd.read_csv('cloudrm_mixed.dat',skiprows=53, header=None, delimiter = ' ', skipinitialspace = True ,index_col = 0)
df3 = pd.read_csv('cloudrm_ice.dat',skiprows=53, header=None, delimiter = ' ', skipinitialspace = True ,index_col = 0)
df4 = pd.read_csv('cloudrm_water.dat',skiprows=53, header=None, delimiter = ' ', skipinitialspace = True ,index_col = 0)
df = pd.concat([df1, df2, df3, df4], axis=0)
df.columns = ['Cloudy 1', 'Cloudy 2', 'Cloudy 3', 'Cloudy 4', 'Cloudy 5', 'Cloudy 6', 'Cloudy 7', 'Cloudy 8', \
              'Cloudy 9', 'Cloudy 10', 'Cloudy 11', 'Cloudy 12', 'Cloudy 13', 'Clear 1', 'Clear 2', 'Clear 3', \
              'Clear 4', 'Clear 5', 'Clear 6', 'Clear 7', 'Clear 8', 'Clear 9', 'Clear 10', 'Clear 11', 'Clear 12', \
              'Clear 13', 'Sat Zen', 'Sun Zen', 'Azi diff', 'COT', 'Ctype', 'Prof ID', 'GOT', 'VIWV' , 'Surf']

In [3]:
#Divide into X and y
variables = ['Cloudy 2', 'Cloudy 3', 'Cloudy 4', 'Cloudy 5', 'Cloudy 6', 'Cloudy 7', 'Cloudy 8', \
              'Cloudy 9', 'Cloudy 10', 'Cloudy 11', 'Cloudy 12', 'Cloudy 13', 'Sat Zen', 'Sun Zen', 'Azi diff']
targets = ['Clear 2', 'Clear 3', 'Clear 4', 'Clear 5', 'Clear 6', 'Clear 7', 'Clear 8', 'Clear 9', 'Clear 10', \
           'Clear 11', 'Clear 12', 'Clear 13', 'COT']

X = df[variables]
y = df[targets]

#Load into tensorflow dataset
tf_dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient="list"), y.to_dict(orient="list")))

tf_dataset


<_TensorSliceDataset element_spec=({'Cloudy 2': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 3': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 4': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 5': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 6': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 7': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 8': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 9': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 10': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 11': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 12': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cloudy 13': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Sat Zen': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Sun Zen': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Azi diff': TensorSpec(shape=(), dtype=tf.float32, name=None)}, {'Clear 2': Tens

In [4]:
def get_train_and_test_splits(dataset,train_size, batch_size=1):
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=500).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [5]:
hidden_units = [8, 8]
learning_rate = 0.001

def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [6]:
FEATURE_NAMES = ['Cloudy 2', 'Cloudy 3', 'Cloudy 4', 'Cloudy 5', 'Cloudy 6', 'Cloudy 7', 'Cloudy 8', \
              'Cloudy 9', 'Cloudy 10', 'Cloudy 11', 'Cloudy 12', 'Cloudy 13', 'Sat Zen', 'Sun Zen', 'Azi diff']


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

In [7]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="relu")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
dataset_size = 200000
batch_size = 32
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(tf_dataset,train_size, batch_size)
train_dataset

<_BatchDataset element_spec=({'Cloudy 2': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 3': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 4': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 5': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 6': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 7': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 8': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 9': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 10': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 11': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 12': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Cloudy 13': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Sat Zen': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Sun Zen': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'Azi diff': Tens

In [10]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
     

In [9]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

NameError: name 'baseline_model' is not defined

In [11]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [12]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilistic output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 100
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
1329/1329 [==============================] - 11s 6ms/step - loss: 272.3416 - root_mean_squared_error: 18.7189 - val_loss: 70.0832 - val_root_mean_squared_error: 18.4725
Epoch 2/100
1329/1329 [==============================] - 6s 4ms/step - loss: 42.3059 - root_mean_squared_error: 17.7305 - val_loss: 28.5231 - val_root_mean_squared_error: 17.5361
Epoch 3/100
1329/1329 [==============================] - 6s 4ms/step - loss: 20.6584 - root_mean_squared_error: 16.8902 - val_loss: 16.1879 - val_root_mean_squared_error: 16.8149
Epoch 4/100
1329/1329 [==============================] - 6s 4ms/step - loss: 12.8539 - root_mean_squared_error: 16.1259 - val_loss: 10.8886 - val_root_mean_squared_error: 16.0253
Epoch 5/100
1329/1329 [==============================] - 7s 5ms/step - loss: 9.0900 - root_mean_squared_error: 15.5203 - val_loss: 8.0594 - val_root_mean_squared_error: 15.5032
Epoch 6/100
1329/1329 [==============================] - 7s 5ms/step - loss: 

1329/1329 [==============================] - 5s 4ms/step - loss: 2.4693 - root_mean_squared_error: 6.5301 - val_loss: 2.1836 - val_root_mean_squared_error: 5.7671
Epoch 48/100
1329/1329 [==============================] - 5s 4ms/step - loss: 2.4674 - root_mean_squared_error: 6.5573 - val_loss: 2.2172 - val_root_mean_squared_error: 5.8995
Epoch 49/100
1329/1329 [==============================] - 6s 4ms/step - loss: 2.4931 - root_mean_squared_error: 6.6464 - val_loss: 2.2139 - val_root_mean_squared_error: 6.1306
Epoch 50/100
1329/1329 [==============================] - 6s 4ms/step - loss: 2.4608 - root_mean_squared_error: 6.5759 - val_loss: 2.1778 - val_root_mean_squared_error: 5.8644
Epoch 51/100
1329/1329 [==============================] - 7s 5ms/step - loss: 2.4751 - root_mean_squared_error: 6.4798 - val_loss: 2.2061 - val_root_mean_squared_error: 5.8513
Epoch 52/100
1329/1329 [==============================] - 6s 4ms/step - loss: 2.4920 - root_mean_squared_error: 6.5866 - val_loss: 2.

1329/1329 [==============================] - 5s 4ms/step - loss: 2.4285 - root_mean_squared_error: 6.1742 - val_loss: 2.1139 - val_root_mean_squared_error: 5.2585
Epoch 94/100
1329/1329 [==============================] - 6s 4ms/step - loss: 2.4228 - root_mean_squared_error: 6.1558 - val_loss: 2.1150 - val_root_mean_squared_error: 5.2239
Epoch 95/100
1329/1329 [==============================] - 5s 4ms/step - loss: 2.4237 - root_mean_squared_error: 6.1726 - val_loss: 2.0935 - val_root_mean_squared_error: 5.2079
Epoch 96/100
1329/1329 [==============================] - 6s 5ms/step - loss: 2.4098 - root_mean_squared_error: 6.0026 - val_loss: 2.1028 - val_root_mean_squared_error: 5.0957
Epoch 97/100
1329/1329 [==============================] - 5s 4ms/step - loss: 2.4041 - root_mean_squared_error: 6.0018 - val_loss: 2.1161 - val_root_mean_squared_error: 5.1960
Epoch 98/100
1329/1329 [==============================] - 5s 4ms/step - loss: 2.4210 - root_mean_squared_error: 6.0755 - val_loss: 2.

In [16]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )
    

Prediction mean: 30.77, stddev: 8.22, 95% CI: [46.88 - 14.67] - Actual: 28.495
Prediction mean: 1.55, stddev: 0.98, 95% CI: [3.48 - -0.37] - Actual: 0.111
Prediction mean: 32.16, stddev: 8.19, 95% CI: [48.21 - 16.1] - Actual: 28.475
Prediction mean: 26.27, stddev: 7.04, 95% CI: [40.07 - 12.47] - Actual: 24.654
Prediction mean: 46.11, stddev: 7.24, 95% CI: [60.3 - 31.92] - Actual: 49.904
Prediction mean: 4.61, stddev: 1.55, 95% CI: [7.65 - 1.57] - Actual: 4.189
Prediction mean: 15.16, stddev: 3.36, 95% CI: [21.75 - 8.57] - Actual: 13.089
Prediction mean: 15.07, stddev: 3.34, 95% CI: [21.61 - 8.53] - Actual: 13.446
Prediction mean: 34.52, stddev: 8.1, 95% CI: [50.38 - 18.65] - Actual: 49.978
Prediction mean: 31.45, stddev: 8.26, 95% CI: [47.64 - 15.26] - Actual: 35.389
